In [1]:
import pandas as pd
import duckdb
import os

In [2]:
base_dir = "data/"
csv_files = [file for file in os.listdir(base_dir) if file.endswith(".csv")]

In [3]:
con = duckdb.connect(database=':memory:', read_only=False)

In [4]:
# Create a table by reading the first CSV file
con.execute(f"CREATE TABLE merged_data AS SELECT * FROM read_csv_auto('{base_dir + csv_files[0]}')")

# Append the remaining CSV files to the table
for file in csv_files[1:]:
    con.execute(f"INSERT INTO merged_data SELECT * FROM read_csv_auto('{base_dir + file}')")

In [5]:
# Example: Get the number of rows in the merged table
row_count = con.execute("SELECT COUNT(*) FROM merged_data").fetchone()[0]
print(f"Total rows: {row_count}")

# Example: Get the column names
columns = con.execute("SELECT * FROM merged_data LIMIT 0").description
print(f"Columns: {[col[0] for col in columns]}")

# Example: Fetch a sample of data as a Pandas DataFrame
sample_df = con.execute("SELECT * FROM merged_data LIMIT 10").df()
print("Sample data:")
print(sample_df)

Total rows: 8031455
Columns: ['Number', 'Start date', 'Start station number', 'Start station', 'End date', 'End station number', 'End station', 'Bike number', 'Bike model', 'Total duration', 'Total duration (ms)']
Sample data:
      Number        Start date Start station number  \
0  130788021  2023-05-21 23:59               003465   
1  130788022  2023-05-21 23:59               003483   
2  130788019  2023-05-21 23:58               001144   
3  130788020  2023-05-21 23:58               200252   
4  130788017  2023-05-21 23:57               001007   
5  130788018  2023-05-21 23:57               000982   
6  130788013  2023-05-21 23:56               001138   
7  130788014  2023-05-21 23:56               200099   
8  130788015  2023-05-21 23:56               200250   
9  130788016  2023-05-21 23:56               001138   

                     Start station          End date End station number  \
0        Eccleston Place, Victoria  2023-05-22 00:17             000990   
1      Fashion St

In [6]:
schema = con.execute("PRAGMA table_info('merged_data')").fetchdf()
print(schema)

    cid                  name     type  notnull dflt_value     pk
0     0                Number   BIGINT    False       None  False
1     1            Start date  VARCHAR    False       None  False
2     2  Start station number  VARCHAR    False       None  False
3     3         Start station  VARCHAR    False       None  False
4     4              End date  VARCHAR    False       None  False
5     5    End station number  VARCHAR    False       None  False
6     6           End station  VARCHAR    False       None  False
7     7           Bike number  VARCHAR    False       None  False
8     8            Bike model  VARCHAR    False       None  False
9     9        Total duration  VARCHAR    False       None  False
10   10   Total duration (ms)   BIGINT    False       None  False


In [7]:
# Find rows where "Start station number" or "End station number" is not an integer
non_integer_rows = con.execute("""
    SELECT *
    FROM merged_data
    WHERE REGEXP_MATCHES(CAST("Start station number" AS VARCHAR), '\\D')
       OR REGEXP_MATCHES(CAST("End station number" AS VARCHAR), '\\D')
""").df()

# Create a new DataFrame without the non-integer rows
cleaned_data = con.execute("""
    SELECT *
    FROM merged_data
    WHERE NOT REGEXP_MATCHES(CAST("Start station number" AS VARCHAR), '\\D')
       AND NOT REGEXP_MATCHES(CAST("End station number" AS VARCHAR), '\\D')
""").df()

print("Rows with non-integer 'Start station number' or 'End station number':")
print(non_integer_rows)

print("\nCleaned data:")
print(cleaned_data)

Rows with non-integer 'End station number':
        Number        Start date Start station number  \
0    129236430  2023-03-16 15:06               300078   
1    129234587  2023-03-16 13:21               200219   
2    129230731  2023-03-16 09:38               200169   
3    129230391  2023-03-16 09:26           200217old2   
4    129226126  2023-03-16 07:59               200091   
..         ...               ...                  ...   
124  129059779  2023-03-07 20:39               300045   
125  129056549  2023-03-07 18:36               001220   
126  129053495  2023-03-07 17:33               200001   
127  129050752  2023-03-07 15:57               200204   
128  129049570  2023-03-07 14:33               300078   

                           Start station          End date End station number  \
0                    Oxford Road, Putney  2023-03-16 15:07         200217old2   
1                Osiers Road, Wandsworth  2023-03-16 13:26         200217old2   
2             Ashley Crescen